In [129]:
import numpy as np
import cv2
import os
import uuid
import math
from IPython.core.debugger import Tracer
from scipy import misc

In [130]:
# INITIALIZATION

In [131]:
video_filename = 'archive/data2/input_video/800x450/IMG_0025.avi'
label_filename = 'archive/data2/video_for_segmentation/IMG_0025.txt'

fps = 25
frame_num = 20
show_width = 800
show_height = 450
fix_frame_num = 80
center_width = show_width / 2
center_height = show_height / 2

with open(label_filename) as f:
    content = f.readlines()

cap = cv2.VideoCapture(video_filename)

frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
label_result = [-1] * frame_count
# fps = cap.get(cv2.CAP_PROP_FPS)

internal = 1000 / fps
internal_ary = [int(internal), int(internal * 2), int(internal * 4)]
internal_idx = 0
frame_idx = 0

name = 'demo_video.avi'
font = cv2.FONT_HERSHEY_SIMPLEX
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(name, fourcc, 25, (show_width, show_height))

In [132]:
def insert_title(frame, title_idx):
    title_img = misc.imread("title_" + str(title_idx) + ".png", mode='RGB')
    height, width, _ = title_img.shape
    convert_title = np.zeros((height, width, 3), dtype=np.uint8)
    convert_title[:,:,0] = title_img[:,:,2]
    convert_title[:,:,1] = title_img[:,:,1]
    convert_title[:,:,2] = title_img[:,:,0]
    start_height = center_height - height / 2
    start_width = center_width - width / 2
    frame[start_height:start_height + height, start_width:start_width + width, :] = convert_title
    return frame

In [133]:
# TITLE

In [134]:
frame = np.zeros((show_height, show_width, 3), dtype=np.uint8)
frame = insert_title(frame, 1)
for i in range(fix_frame_num):
    out.write(frame)
for i in range(fix_frame_num / 2):
    frame = frame * 0.9
    frame = frame.astype(np.uint8)
    out.write(frame)

In [135]:
# ACTION RECOGNITION AND SHARPNESS CALCULATION

In [136]:
text = ["none", "Staring", "Paging", "shake", "else"]
cap = cv2.VideoCapture(video_filename)

frame = np.zeros((show_height, show_width, 3), dtype=np.uint8)
out.write(frame)

frame = insert_title(frame, 2)
for i in range(fix_frame_num):
    out.write(frame)

while cap.isOpened():
    frame_idx = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
    ret, frame = cap.read()
    if ret != True:
        break
    frame = cv2.resize(frame, (show_width, show_height), interpolation = cv2.INTER_AREA)
    blurry = cv2.Laplacian(frame, cv2.CV_64F).var()
    cv2.putText(frame, "Sharpness: {:.2f}".format(blurry), (10, 40),
        font, 1, (0, 0, 255), 2)
    if content[0][frame_idx] != '-':
        cv2.putText(frame, text[int(content[0][frame_idx])], (show_width - 200, show_height - 20), font, 1, (0, 0, 255), 2)
    cv2.putText(frame, str(frame_idx) + '/' + str(frame_count), (10, show_height - 20), font, 1, (0,0,255), 2)
    out.write(frame)

cap.release()
cv2.destroyAllWindows()

In [137]:
# FIND CONTENT AREA FROM ONE FRAME

In [138]:
frame = np.zeros((show_height, show_width, 3), dtype=np.uint8)
frame = insert_title(frame, 3)
for i in range(fix_frame_num):
    out.write(frame)

In [139]:
filename_ary = ["4c5898ca-be6f-11e6-9959-d017c2cf90d9", "4ca99aae-be6f-11e6-9959-d017c2cf90d9", "4d8ed5c4-be6f-11e6-9959-d017c2cf90d9"]

for filename in filename_ary:
    ori_image_path = "archive/data2/images/800x450/" + filename + ".png"
    mask_image_path = "archive/data2/images/800x450_mask/" + filename + ".png"
    ori_image = misc.imread(ori_image_path, mode='RGB') / 2
    mask_image = misc.imread(mask_image_path, mode='RGB')
    for i in range(show_height):
        for j in range(show_width):
            if mask_image[i, j, 0] != 255:
                ori_image[i, j, 0] = ori_image[i, j, 0] + 123
                ori_image[i, j, 1] = ori_image[i, j, 1]
                ori_image[i, j, 2] = ori_image[i, j, 2]

    for i in range(fix_frame_num):
        out.write(ori_image)

In [140]:
# FIND LINES FROM CONTENT AREA

In [141]:
frame = np.zeros((show_height, show_width, 3), dtype=np.uint8)
frame = insert_title(frame, 4)

for i in range(fix_frame_num):
    out.write(frame)

In [142]:
for i in range(3):
    frame = misc.imread("find_line_" + str(i) + ".png", mode='RGB')
    for i in range(fix_frame_num):
        out.write(frame)

In [143]:
# TEXT RECOGNITION

In [144]:
frame = np.zeros((show_height, show_width, 3), dtype=np.uint8)
frame = insert_title(frame, 5)

for i in range(fix_frame_num):
    out.write(frame)


frame = np.zeros((show_height, show_width, 3), dtype=np.uint8)
height_base = 50
height_interval = 90
width_base = 200
text_width_base = 15
text_interval = 30
for i in range(5):
    text_img = misc.imread("line_recog_" + str(i) + ".png", mode="RGB")
    height, width, _ = text_img.shape
    cur_h_base = height_base + i * height_interval
    frame[cur_h_base:cur_h_base + height, width_base:width_base + width, :] = text_img
    with open("line_recog_" + str(i) + ".txt") as f:
        content = f.readlines()
    # you may also want to remove whitespace characters like `\n` at the end of each line
    content = [x.strip() for x in content]
    cv2.putText(frame, content[0], (text_width_base, cur_h_base + text_interval), font, 0.34, (0, 255, 255), 1)
for i in range(fix_frame_num * 2):
    out.write(frame)



In [145]:
# THE END

In [146]:
frame = np.zeros((show_height, show_width, 3), dtype=np.uint8)
frame = insert_title(frame, 6)
for i in range(fix_frame_num / 2):
    out.write(frame)
for i in range(fix_frame_num / 2):
    frame = frame * 0.9
    frame = frame.astype(np.uint8)
    out.write(frame)

In [147]:
out.release()